In [ ]:
# whenever raw data files have been altered, run "../scripts/xml_cleaner.py"

# Algorithm for extracting data from nested XML files (pre-cleaned)

In [1]:
import xml.etree.ElementTree as ET
import pandas as pd

# custom function for extracting data from xml and saving to pandas DataFrame
def patent_xml2df(xml_data):
    root = ET.XML(xml_data)
    all_records = []
    for patent in root:
        record = {}
        for element in patent[0]:
            if element.tag == 'publication-reference':
                for item in element[0]:
                    if item.tag == 'date':
                        record['grant_date'] = item.text
                    elif item.tag == 'doc-number':
                        record['patent_number'] = item.text
            elif element.tag == 'application-reference':
                record['application_type'] = element.get('appl-type')
                for item in element[0]:
                    if item.tag == 'date':
                        record['filing_date'] = item.text
                    elif item.tag == 'doc-number':
                        record['application_number'] = item.text
            elif element.tag == 'figures':
                for item in element:
                    record[item.tag] = item.text
            elif element.tag == 'us-parties':
                address_book = element[0][0][0]
                for item in address_book:
                    if item.tag == 'address':
                        for info in item:
                            record[info.tag] = info.text
            elif element.tag == 'number-of-claims':
                record['number_of_claims'] = element.text
            elif element.tag == 'invention-title':
                if element.text == None:
                    record['invention_title'] = element[0].text + element[0].tail
                else:
                    record['invention_title'] = element.text
        all_records.append(record)
    return pd.DataFrame(all_records)

# Using globbing to get names of all cleaned XML file paths &
# compiling list of DataFrames for concatenation

In [2]:
import glob

# use globbing to get all names of cleaned xml file paths
clean_files = glob.glob('../clean_data/*')


# loop over file names, extract data and concatenate into single DataFrame
xml_data_list = []
for clean_file in clean_files:
    data = open(clean_file).read()
    df = patent_xml2df(data)
    xml_data_list.append(df)
    
patents_df = pd.concat(xml_data_list, ignore_index=True)
    

In [3]:
patents_df.head()

,application_number,application_type,city,country,filing_date,grant_date,invention_title,number-of-drawing-sheets,number-of-figures,number_of_claims,patent_number,state
0,29541102,design,Minneapolis,US,20150930,20171031,Shaped tortilla,3,6,1,D0800990,MN
1,29539276,design,Atlanta,US,20150911,20171031,Utility belt,7,9,1,D0800991,GA
2,29555649,design,Baltimore,US,20160224,20171031,Brassiere,7,7,1,D0800992,MD
3,29519436,design,Karmiel,IL,20150305,20171031,Shirt,4,4,1,D0800993,NaN
4,35500692,design,Bayreuth,DE,20150908,20171031,Footmuff for a stroller for children,7,7,1,D0800994,NaN


# Casting Data Types for Specific Columns

In [4]:
# change the "filing_date" and "grant_date" columns to datetime objects
patents_df['filing_date'] = pd.to_datetime(patents_df['filing_date'])
patents_df['grant_date'] = pd.to_datetime(patents_df['grant_date'])

# change number-of-claims column to type int
patents_df['number_of_claims'] = patents_df['number_of_claims'].astype(int)

# Filling Null Values 

### Filling with '0' for scalar values (i.e., these values are 'NaN' in raw XML when value is zero)

In [5]:
patents_df['number-of-drawing-sheets'] = patents_df['number-of-drawing-sheets'].fillna('0')
patents_df['number-of-figures'] = patents_df['number-of-figures'].fillna('0')

### Filling NaN values in address values (`country`, `state` and `city`) with 'missing'

In [9]:
patents_df['state'] = patents_df['state'].fillna('missing')
patents_df['country'] = patents_df['country'].fillna('missing')
patents_df['city'] = patents_df['city'].fillna('missing')

# Conclusions:  'Null' values

There are 12633 patent grants that do not contain figures or drawings (and thus have zero drawing sheets).

Many of the patent applicants are located outside the US and therefore have no `state` value in the XML.  In addition, some patent records simply don't include an address for the applicant.  For those patents, the Null address values (`country`, `state` and `city`) were replaced with 'missing' for clarification.

The `invention_title` elements in the raw XML files often contained nested styling tags (e.g., the italics tag &lt;i&gt; was used for genus names of plants and other living organisms).

In [10]:
patents_df.to_csv('../intermediate_csv_files/clean_raw_data.csv')